### Connecting to a SQL Database Project

In [3]:
import os
import pandas as pd
from sqlalchemy import create_engine, text
from dotenv import load_dotenv

#### Part 1: Install Dependencies

Run the command pip install -r requirements.txt

#### Part 2: Add Your Database Credentials

DB_USER='remelop'
DB_PASSWORD='Melo0316'
DB_PORT= 3306
DB_HOST='localhost'
DB_NAME='exercise'

#### Part 3 : Create a Database Using PostgreSQL

Make sure you have installed the Postgres client for the terminal called PSQL by running the following command:

$ psql --version

1. Postgres is a database server, you have to start the server before you can use it. In order to stat postgres in your computer run the following command:

sudo service postgresql start

2. Create a new user to connect to your database:

psql -U postgres -c "CREATE USER remelop;"

3. Create a new database within the Postgres engine by customizing and executing the following command:

psql -U postgres -c "CREATE DATABASE exercise OWNER remelop;"

4. Connect to the Postgres engine to use your database, manipulate tables and data:

psql -h localhost -U remelop exercise

#### Part 4: Understand the structure of your project's template

Once you have completed the above steps, you will have created your SQL database, but there are no tables yet. Next, you need to connect to your empty database, and you will create some tables using Python via SQL scripts that you will find in the ./src/sql/ folder.